<a href="https://colab.research.google.com/github/KucharskiR/data-science/blob/main/trained/001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Sieć neuronowa LSTM**

Budowa sieci neuronowej LSTM

#### **Spis treści:**
1. [Import bibliotek](#a0)
2. [Przygotowanie danych](#a1)
3. [Konfiguracja LSTM layer](#a2)
4. [Główny model](#a3)
5. [Klasyfikacja](#a4)
6. [Przykład budowy modelu z kursu](#a5)
7. [Ocena modelu + wykresy](#a6)
8. [Predykcja na podstawie modelu](#a7)
9. [Zip file](#a8)
10. [Extract .tar.gz](#a9)

### <a name='a0'></a> Import bibliotek

In [1]:
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import LambdaCallback
from keras.initializers import TruncatedNormal
from keras.models import save_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
import os

np.set_printoptions(precision=12, suppress=True, linewidth=150)
pd.options.display.float_format = '{:.6f}'.format
# sns.set()   <--- seaborn
print(np.__version__)

1.25.2


### <a name='a1'></a> Przygotowanie danych

In [256]:
# Epochs
epochs = 5

In [257]:
# Assuming there are three features in your data
num_labels = 2
timestepsPerSample = 150
batch = 32

In [4]:
# Replace 'your_file.csv' with the actual file path
file_features = './1_150x9/1_150x9f.csv'
file_labels = './1_150x9/1_150x9l.csv'

# Specify the CSV file name
csv_file_name = 'my_data1.csv'

# Read the .csv file and create an array
data_strings = np.genfromtxt(file_features, delimiter=';')
labels_strings = np.genfromtxt(file_labels,delimiter=';')

In [258]:
# Wycinanie wybranych kolumn
# data_s = data_strings[:,[0,2,3,4,5,6,7,8]]
data_s = data_strings[:,[0,4,5,8]]
num_features = data_s.shape[1]
print(data_strings[:3])
print(data_s[:3])

[[-0.07371   0.001137  0.01742  -0.0145   -0.06481   0.08029  -0.11523   0.10792   0.989   ]
 [-0.17048   0.007527  0.019168 -0.01685  -0.10209  -0.44286  -0.18144   0.01512   0.94915 ]
 [-0.17888   0.007856  0.018207 -0.01051  -0.09147  -0.71242  -0.25967  -0.04955   0.76432 ]]
[[-0.07371 -0.06481  0.08029  0.989  ]
 [-0.17048 -0.10209 -0.44286  0.94915]
 [-0.17888 -0.09147 -0.71242  0.76432]]


In [259]:
# Convert from strings to float and int
X = data_s.astype(float).reshape((-1,timestepsPerSample,num_features))
Y = labels_strings.astype(float).reshape((-1,3))
print(X.shape)
print(Y.shape)

(33300, 150, 4)
(33300, 3)


In [260]:
# Modification from imported to new size
X_mod = X[:10000,130:]
Y_mod = Y[:10000]
timestepsPerSample = X_mod.shape[1]
print(X_mod.shape)
print(Y_mod.shape)
print(X_mod[:1])
# print(X[:1])

(10000, 20, 4)
(10000, 3)
[[[-0.10312 -0.03733 -0.73913  0.99668]
  [-0.00252 -0.03436 -0.51402  0.99574]
  [ 0.0215  -0.04084 -0.30769  0.99498]
  [ 0.06953 -0.0703   0.11111  0.99477]
  [-0.00904 -0.09481  0.31579  0.99398]
  [-0.00904 -0.06843  0.34375  0.99347]
  [ 0.08219 -0.09397  0.2549   0.99421]
  [ 0.08219 -0.11162  0.38462  0.99497]
  [ 0.03565 -0.06563  0.5      0.99527]
  [ 0.03565 -0.09871  0.27273  0.99548]
  [ 0.00232 -0.0933  -0.0303   0.99534]
  [-0.01475 -0.09204 -0.25     0.99506]
  [ 0.08586 -0.08117 -0.03448  0.99544]
  [ 0.08586 -0.11801  0.24138  0.99579]
  [ 0.00854 -0.16035  0.33333  0.99566]
  [ 0.02719 -0.1198   0.       0.99559]
  [ 0.02719 -0.11146 -0.22581  0.99556]
  [ 0.00526 -0.00849 -0.31034  0.99546]
  [-0.03893  0.02978 -0.42857  0.99514]
  [-0.0149   0.14187 -0.28571  0.99497]]]


In [305]:
# splitting the dataset 75% for training and 25% testing
# X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
x_train, x_test, Y_train, Y_test = train_test_split(X_mod,Y_mod, test_size=0.15, shuffle=False)
print(x_train.shape)
# print(x_train[:5])

(8500, 20, 4)


In [306]:
y_train = Y_train[:, 0:2]
y_test = Y_test[:, 0:2]
print(y_train[:2])
print(y_test[:2])

[[0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]]


### <a name='a4'></a> Konfiguracja LSTM layer

In [307]:
LstmLayer = LSTM(
    units=600,
    activation="tanh",
    recurrent_activation="sigmoid",
    use_bias=True,
    kernel_initializer="glorot_uniform",
    recurrent_initializer="orthogonal",
    bias_initializer="zeros",
    unit_forget_bias=True,
    kernel_regularizer=None,
    recurrent_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    recurrent_constraint=None,
    bias_constraint=None,
    dropout=0.0,
    recurrent_dropout=0.0,
    seed=None,
    return_sequences=False,
    return_state=False,
    go_backwards=False,
    stateful=False,
    unroll=False,
    input_shape=(x_train.shape[1],x_train.shape[2])
)

### <a name='a3'></a> Główny model

In [308]:
# Optimizer decay
num_samples = x_train.shape[0]
STEPS_PER_EPOCH = num_samples/batch
num_samples
print(f"Timesteps: {timestepsPerSample}")
print(f"Num Samples: {num_samples}")
print(f"Num features: {num_features}")

Timesteps: 20
Num Samples: 8500
Num features: 4


In [309]:
model = Sequential() # initializing model

# input layer and LSTM layer with 50 neurons
# lst.add(LSTM(units=300, return_sequences=False, input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(LstmLayer)
# model.add(Dense(50, activation='relu'))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(20, activation='relu'))
# outpute layer with sigmoid activation
model.add(Dense(num_labels, activation='sigmoid'))

# lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*100,
  decay_rate=0.9,
  staircase=False)

def get_optimizer():
  # return tf.keras.optimizers.Adam(learning_rate=0.001)
  return tf.keras.optimizers.Adam(learning_rate= lr_schedule)

optimizer = get_optimizer()

# defining loss function, optimizer, metrics and then compiling model
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_14 (LSTM)              (None, 600)               1452000   
                                                                 
 dense_47 (Dense)            (None, 2)                 1202      
                                                                 
Total params: 1453202 (5.54 MB)
Trainable params: 1453202 (5.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Training

In [310]:
# training the model on training dataset
# history = lst.fit(x_train, y_train, epochs=epochs, batch_size=batch,validation_split=0.2)
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch, validation_data=(x_test, y_test))

# Save model
model.save("1_150x_trained.h5", overwrite=True, save_format='h5')

# predicting target attribute on testing dataset
predict = model.predict(x_test)
# predict = np.argmax(lst.predict(x_test), axis=-1)

Epoch 1/5
266/266 [==============================] - 73s 261ms/step - loss: 0.6093 - accuracy: 0.6731 - val_loss: 0.5852 - val_accuracy: 0.7053
Epoch 2/5
266/266 [==============================] - 66s 246ms/step - loss: 0.5825 - accuracy: 0.6958 - val_loss: 0.5784 - val_accuracy: 0.6973
Epoch 3/5
266/266 [==============================] - 67s 254ms/step - loss: 0.5734 - accuracy: 0.7040 - val_loss: 0.5853 - val_accuracy: 0.6927
Epoch 4/5
266/266 [==============================] - 70s 263ms/step - loss: 0.5705 - accuracy: 0.7054 - val_loss: 0.5709 - val_accuracy: 0.6993
Epoch 5/5
266/266 [==============================] - 66s 247ms/step - loss: 0.5681 - accuracy: 0.7047 - val_loss: 0.5704 - val_accuracy: 0.7073


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



47/47 [==============================] - 5s 98ms/step


In [311]:
# Set print options to suppress scientific notation
np.set_printoptions(suppress=True)

# Concatenate arrays
result = np.hstack((predict, y_test))
print(result)

test_results = model.evaluate(x_test, y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

# # Create a DataFrame from the 2D array
# df = pd.DataFrame(predict, columns=['Column1', 'Column2'])

# # Export the DataFrame to CSV with semicolon as the delimiter and avoiding scientific notation
# # df.to_csv(csv_file_name, sep=';', index=False, float_format='%.0f')
# df.to_csv(csv_file_name, sep=';', index=False)

# # Read the CSV file into a DataFrame without header
# df = pd.read_csv(csv_file_name, sep=';', header=None)

# # Drop the first row containing data
# df = df.iloc[1:]

# # Save the modified DataFrame back to the CSV file without header
# df.to_csv(csv_file_name, sep=';', index=False, header=False)

# print(f'CSV file name: {csv_file_name}.')



[[0.327067911625 0.676469802856 0.             1.            ]
 [0.379106253386 0.625310659409 0.             1.            ]
 [0.482015907764 0.522036373615 0.             1.            ]
 ...
 [0.781472504139 0.222117513418 1.             0.            ]
 [0.683960258961 0.320381373167 1.             0.            ]
 [0.640357255936 0.36431440711  1.             0.            ]]
47/47 [==============================] - 3s 62ms/step - loss: 0.5704 - accuracy: 0.7073
Test results - Loss: 0.5704068541526794 - Accuracy: 70.73333263397217%


### <a name='a4'></a> Predict i Klasyfikacja

In [312]:
predict = model.predict(x_test)
predict[:3]

47/47 [==============================] - 5s 107ms/step


array([[0.3270679 , 0.6764698 ],
       [0.37910625, 0.62531066],
       [0.4820159 , 0.5220364 ]], dtype=float32)

In [313]:
# np.info(predict)
# np.where(y_pred > threshold, 1,0)
predict_classes = predict
# predict_classes = np.argmax(predict_classes, axis=-1)
predict_classes = np.where(predict > 0.5, 1,0)
np.info(predict_classes)
predict_classes[:3]
# predict

class:  ndarray
shape:  (1500, 2)
strides:  (16, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x5b192c6d9930
byteorder:  little
byteswap:  False
type: int64


array([[0, 1],
       [0, 1],
       [0, 1]])

**epochs** - ile razy zestaw treningowy zostanie przetworzony przez model. Przy każdej iteracji optymalizator próbuje dopasować wagi, aby funkcja celu została zminimalizowana.

**batch_size** - liczba przykładów treningowych po której następuje aktualizacji wag

**validation_split** - procent danych użytych do walidacji

In [314]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics[:3]

,loss,accuracy,val_loss,val_accuracy,epoch
0,0.609282,0.673059,0.585209,0.705333,0
1,0.582526,0.695765,0.578353,0.697333,1
2,0.573397,0.704000,0.585326,0.692667,2


### <a name='a6'></a> Ocena modelu LSTM

In [315]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['accuracy'], name='accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['loss'], name='loss'), row=1, col=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_accuracy'], name='val_accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_loss'], name='val_loss'), row=1, col=2)

fig.update_xaxes(title_text='epochs')
fig.update_yaxes(title_text='accuracy')
fig.update_layout(width=1000, title='Accuracy and Loss')
fig.show()

In [316]:
# nie wiem co to za blok
from keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=10)
y_test_cat = to_categorical(y_test, num_classes=10)

In [317]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(test_acc)

0.7073333263397217


### <a name='a7'></a> Graf zielono czerwony

In [318]:
# Połączenie dwóch tablic
concat = np.hstack((predict_classes, Y_test[:,2:]))
print(concat.shape)
print(concat[:3])

(1500, 3)
[[ 0.    1.   85.93]
 [ 0.    1.   85.92]
 [ 0.    1.   85.9 ]]


In [319]:
import plotly.express as px
import pandas as pd

# Tworzenie Date Frame
df = pd.DataFrame(concat, columns=['Sell', 'Buy', 'Price'])
df.head(5)

,Sell,Buy,Price
0,0.000000,1.000000,85.930000
1,0.000000,1.000000,85.920000
2,0.000000,1.000000,85.900000
3,1.000000,0.000000,85.890000
4,1.000000,0.000000,85.990000


In [323]:
import plotly.graph_objects as go
import pandas as pd

# Tworzenie wykresu liniowego
fig = go.Figure()

start = 500
end = start + 500

# Dodawanie linii do wykresu
for i in range(start, len(df[:end])):
    color = 'green' if df.at[i, 'Buy'] == 1 else 'red'
    fig.add_trace(go.Scatter(x=[i-1, i], y=[df.at[i-1, 'Price'], df.at[i, 'Price']], line=dict(color=color), showlegend=False))

# Ustawienia osi i tytuł
fig.update_layout(
    xaxis_title='Indeks',
    yaxis_title='Close',
    title='Wykres liniowy z kolorami'
)

# Wyświetlanie wykresu
fig.show()

In [321]:
# Profit
sum = 0;
for i in range(0, len(df)):
  if ((i-1) > 0):
    if df.at[i,'Sell'] == 1:
      sum += (df.at[i,'Price'] - df.at[i-1,'Price'])*(-1)
    elif df.at[i,'Buy'] == 1:
      sum += (df.at[i,'Price'] - df.at[i-1,'Price'])
    if df.at[i,'Sell'] != df.at[i-1,'Sell']:
      sum -= 0.03

print(sum)

-4.029999999999849


### <a name='a7'></a> Predykcja na podstawie modelu:



1.   **model.evaluate(y_true, y_pred)** - pozwala obliczyć metryki modelu
2.   **model.predict_classes()** - pozwala zwrócić odpowiednio przewidziane klasy
3.   **model.predict_proba(), model.predict()** - pozwala zwrócić prawdopodobieństwo danej klasy





In [ ]:
predictions = model.predict(x_test)
predictions

In [ ]:
# predictions_cls = model.predict_classes(X_test)
predictions_cls = np.argmax(model.predict(x_test), axis=-1)
predictions_cls

In [ ]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics

In [ ]:
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
model = tf.keras.models.load_model('lstm_Model.keras')
model.summary()

### <a name='a5'></a> Przykład budowy modelu z kursu

In [ ]:
# model = Sequential()
# model.add(Flatten(input_shape=(28, 28)))
# model.add(Dense(units=128, activation='relu'))
# model.add(Dense(units=10, activation='softmax'))

# model.compile(optimizer='rmsprop',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# model.summary()
model = Sequential()
model.add(LstmLayer)
model.add(Dense(units=2, activation='softmax')) # <----- output layer

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

### <a name='a8'></a> Zip file

In [ ]:
from zipfile import ZipFile

zip = ZipFile('my_python_files.zip','w')
zip.write('lstm_Model.keras')

### <a name='a9'></a> Extract tar gz

In [2]:
# importing the "tarfile" module
import tarfile

# open file
file = tarfile.open('1_150x9.tar.gz')

# extracting a specific file
file.extractall(path='./1_150x9/')

file.close()